In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


<h2> <b>Задача:</b> Найти более подходящие промпты в языковые модели для создания запросов в браузер по определённной тематике</h2>

```
{
  "example-prompt": "создай запрос в браузер для поиска аналитики финансового рынка за 2 квартал 2023 года. Без лишней информации, только запрос"
}
```
<h3>Приблизительный ответ <i>(я использую phind, API которого не получится использовать для нашего конечного решения)</i>:</h3>
<p><b>Аналитика финансового рынка Q2 2023</b></p>


In [2]:
!pip install bs4

In [3]:
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [4]:
from bs4 import BeautifulSoup

with open('drive/MyDrive/economy_research.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

text = soup.get_text()

In [5]:
# Исключительно ознакомительный пример.
# НЕ НАДО ТАК ИНФЕРИТЬ МОДЕЛЬ В ПРОДЕ.
# Когда есть https://github.com/vllm-project/vllm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
%%time
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

query = f'Создай json для диаграммы и таблицу с информацией об экономике России, используя данные из этой таблицы {text}'
prompt = tokenizer.apply_chat_template([{
    "role": "system",
    "content": DEFAULT_SYSTEM_PROMPT
}, {
    "role": "user",
    "content": query
}], tokenize=False, add_generation_prompt=True)
data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
data = {k: v.to(model.device) for k, v in data.items()}
output_ids = model.generate(**data, generation_config=generation_config)[0]
output_ids = output_ids[len(data["input_ids"][0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
print(query)
print(output)
print()
print("==============================")
print()

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Создай json для диаграммы и таблицу с информацией об экономике России, используя данные из этой таблицы 






















































Вечерний обзор финансовых рынков. Уходящий 2023 год — один из лучших для российского рынка - Аналитика. АРБ: Ассоциация российских банков











АКИБАНК
Альфа-Банк
Банк ЗЕНИТ
Банк Интеза
БИНБАНК Столица
ВЕК
ВТБ24
МСП Банк
Ренессанс Банк
Руснарбанк
Сбербанк
Совкомбанк
Трансстройбанк






ОБ АССОЦИАЦИИ  




Общие сведения


Контактная информация


Управление


Члены АРБ


Проекты АРБ


Новости АРБ


Издания АРБ


АРБ и госвласть


Всероссийский банковский совет


Работа комитетов


Национальный банковский клуб


Кодекс этических принципов банковского дела


Стандарт потребителям финансовых услуг


Фотоархив


Видеоархив

Архив

Конференции и форумы


30-летие АРБ






Про банки  




Новости финансового сектора


Горячая тема


От первого лица


Открытые дискуссии


Календарь мероприятий


Аналитика


Работа с документами


